# Caltech-101 Classification: Model Comparison and Results Analysis

This notebook analyzes and compares the performance of different models:
- Classical ML: HOG + SVM
- Deep Learning: ResNet, EfficientNet, Vision Transformer (ViT)

Analysis includes:
- Performance metrics comparison
- Confusion matrices
- Per-class accuracy
- Training curves
- Ablation study results

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Load Evaluation Metrics

In [ ]:
# Load evaluation summary
results_dir = Path('../results')
metrics_dir = results_dir / 'metrics'

# Find all metrics files
metrics_files = list(metrics_dir.glob('*_test_metrics.json'))

# Load all metrics
all_metrics = {}
for metrics_file in metrics_files:
    model_name = metrics_file.stem.replace('_test_metrics', '')
    with open(metrics_file, 'r') as f:
        all_metrics[model_name] = json.load(f)

print(f"Loaded metrics for {len(all_metrics)} models:")
for model_name in all_metrics.keys():
    print(f"  - {model_name}")

## 2. Overall Performance Comparison

In [ ]:
# Extract main metrics
metric_names = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
model_names = list(all_metrics.keys())

# Create comparison table
print("\n" + "="*100)
print(f"{'Model':<25}{'Accuracy':<15}{'Precision':<15}{'Recall':<15}{'F1-Score':<15}")
print("-"*100)

for model in model_names:
    metrics = all_metrics[model]
    print(f"{model:<25}{metrics['accuracy']:<15.4f}{metrics['precision_macro']:<15.4f}"
          f"{metrics['recall_macro']:<15.4f}{metrics['f1_macro']:<15.4f}")

print("="*100)

# Find best model
best_model = max(all_metrics.items(), key=lambda x: x[1]['accuracy'])
print(f"\n🏆 Best Model: {best_model[0]} (Accuracy: {best_model[1]['accuracy']:.4f})")

In [ ]:
# Visualize comparison
n_models = len(model_names)
n_metrics = len(metric_names)

data = np.zeros((n_models, n_metrics))
for i, model in enumerate(model_names):
    for j, metric in enumerate(metric_names):
        if metric in all_metrics[model]:
            data[i, j] = all_metrics[model][metric]

# Plot grouped bar chart
fig, ax = plt.subplots(figsize=(14, 7))

x = np.arange(n_metrics)
width = 0.8 / n_models

colors = plt.cm.Set3(np.linspace(0, 1, n_models))

for i, model in enumerate(model_names):
    offset = (i - n_models / 2) * width + width / 2
    bars = ax.bar(x + offset, data[i], width, label=model, alpha=0.8, color=colors[i])
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=8)

ax.set_xlabel('Metrics', fontsize=13, fontweight='bold')
ax.set_ylabel('Score', fontsize=13, fontweight='bold')
ax.set_title('Model Performance Comparison', fontsize=15, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(metric_names, fontsize=11)
ax.legend(fontsize=10, loc='lower right')
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, 1.0)

plt.tight_layout()
plt.show()

## 3. Per-Class Performance Analysis

In [ ]:
# Select best model for detailed analysis
best_model_name = best_model[0]
best_metrics = all_metrics[best_model_name]

if 'per_class_metrics' in best_metrics:
    per_class = best_metrics['per_class_metrics']
    
    # Extract data
    classes = list(per_class.keys())
    accuracies = [per_class[c]['accuracy'] for c in classes]
    f1_scores = [per_class[c]['f1_score'] for c in classes]
    
    # Sort by F1-score
    sorted_indices = np.argsort(f1_scores)
    
    # Plot top 20 best and worst
    n_show = 20
    worst_indices = sorted_indices[:n_show]
    best_indices = sorted_indices[-n_show:]
    selected_indices = np.concatenate([worst_indices, best_indices])
    
    selected_classes = [classes[i] for i in selected_indices]
    selected_f1 = [f1_scores[i] for i in selected_indices]
    
    colors = ['red'] * n_show + ['green'] * n_show
    
    fig, ax = plt.subplots(figsize=(12, 12))
    y_pos = np.arange(len(selected_classes))
    ax.barh(y_pos, selected_f1, color=colors, alpha=0.6)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(selected_classes, fontsize=9)
    ax.set_xlabel('F1-Score', fontsize=12, fontweight='bold')
    ax.set_title(f'{best_model_name}: Top {n_show} Best and Worst Classes', 
                fontsize=14, fontweight='bold', pad=15)
    ax.axvline(x=np.mean(f1_scores), color='blue', linestyle='--', 
              linewidth=2, label=f'Mean: {np.mean(f1_scores):.3f}')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()

## 4. Training History Analysis

In [ ]:
# Load training histories (from model checkpoints if available)
import torch

models_dir = Path('../models')
checkpoint_files = list(models_dir.glob('*_best.pth'))

histories = {}
for ckpt_file in checkpoint_files:
    model_name = ckpt_file.stem.replace('_best', '')
    try:
        checkpoint = torch.load(ckpt_file, map_location='cpu')
        if 'history' in checkpoint:
            histories[model_name] = checkpoint['history']
    except:
        pass

if histories:
    # Plot training curves
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(histories)))
    
    for i, (model_name, history) in enumerate(histories.items()):
        epochs = range(1, len(history['train_loss']) + 1)
        
        # Loss
        axes[0].plot(epochs, history['train_loss'], '-', 
                    color=colors[i], alpha=0.7, label=f'{model_name} (train)', linewidth=2)
        axes[0].plot(epochs, history['val_loss'], '--', 
                    color=colors[i], alpha=0.7, label=f'{model_name} (val)', linewidth=2)
        
        # Accuracy
        axes[1].plot(epochs, history['train_acc'], '-', 
                    color=colors[i], alpha=0.7, label=f'{model_name} (train)', linewidth=2)
        axes[1].plot(epochs, history['val_acc'], '--', 
                    color=colors[i], alpha=0.7, label=f'{model_name} (val)', linewidth=2)
    
    axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Loss', fontsize=12, fontweight='bold')
    axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold', pad=15)
    axes[0].legend(fontsize=9, loc='upper right')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
    axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold', pad=15)
    axes[1].legend(fontsize=9, loc='lower right')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No training histories found in model checkpoints.")

## 5. Ablation Study Results

In [ ]:
# Load ablation study results
ablation_dir = results_dir / 'ablation'
ablation_summary_path = ablation_dir / 'ablation_summary.json'

if ablation_summary_path.exists():
    with open(ablation_summary_path, 'r') as f:
        ablation_results = json.load(f)
    
    print("\nAblation Study Results:")
    print("="*80)
    
    n_studies = len(ablation_results)
    fig, axes = plt.subplots(1, n_studies, figsize=(6*n_studies, 5))
    
    if n_studies == 1:
        axes = [axes]
    
    for idx, (study_name, results) in enumerate(ablation_results.items()):
        print(f"\n{study_name.upper()}:")
        for exp_name, accuracy in results.items():
            print(f"  {exp_name}: {accuracy:.4f}")
        
        # Plot
        experiments = list(results.keys())
        accuracies = list(results.values())
        
        bars = axes[idx].bar(range(len(experiments)), accuracies, 
                            color='steelblue', alpha=0.7)
        
        # Add value labels
        for bar, acc in zip(bars, accuracies):
            axes[idx].text(bar.get_x() + bar.get_width() / 2., acc,
                          f'{acc:.4f}', ha='center', va='bottom', 
                          fontsize=11, fontweight='bold')
        
        axes[idx].set_xticks(range(len(experiments)))
        axes[idx].set_xticklabels(experiments, rotation=30, ha='right')
        axes[idx].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
        axes[idx].set_title(f'Ablation: {study_name.replace("_", " ").title()}', 
                           fontsize=13, fontweight='bold', pad=15)
        axes[idx].grid(True, alpha=0.3, axis='y')
        axes[idx].set_ylim(0, 1.0)
    
    plt.tight_layout()
    plt.show()
    print("="*80)
else:
    print("No ablation study results found.")

## 6. Classical ML vs Deep Learning Comparison

In [ ]:
# Categorize models
classical_models = {k: v for k, v in all_metrics.items() if 'hog' in k.lower() or 'svm' in k.lower()}
deep_models = {k: v for k, v in all_metrics.items() if k not in classical_models}

if classical_models and deep_models:
    # Calculate average metrics for each category
    classical_avg = {
        'accuracy': np.mean([m['accuracy'] for m in classical_models.values()]),
        'f1_macro': np.mean([m['f1_macro'] for m in classical_models.values()])
    }
    
    deep_avg = {
        'accuracy': np.mean([m['accuracy'] for m in deep_models.values()]),
        'f1_macro': np.mean([m['f1_macro'] for m in deep_models.values()])
    }
    
    # Visualize comparison
    categories = ['Classical ML', 'Deep Learning']
    accuracy_values = [classical_avg['accuracy'], deep_avg['accuracy']]
    f1_values = [classical_avg['f1_macro'], deep_avg['f1_macro']]
    
    x = np.arange(len(categories))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars1 = ax.bar(x - width/2, accuracy_values, width, label='Accuracy', 
                   color='skyblue', alpha=0.8)
    bars2 = ax.bar(x + width/2, f1_values, width, label='F1-Score (Macro)', 
                   color='lightcoral', alpha=0.8)
    
    # Add value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2., height,
                   f'{height:.4f}', ha='center', va='bottom', 
                   fontsize=12, fontweight='bold')
    
    ax.set_ylabel('Score', fontsize=13, fontweight='bold')
    ax.set_title('Classical ML vs Deep Learning Comparison', 
                fontsize=15, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=12)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(0, 1.0)
    
    plt.tight_layout()
    plt.show()
    
    print("\nClassical ML vs Deep Learning:")
    print(f"  Classical ML - Avg Accuracy: {classical_avg['accuracy']:.4f}, Avg F1: {classical_avg['f1_macro']:.4f}")
    print(f"  Deep Learning - Avg Accuracy: {deep_avg['accuracy']:.4f}, Avg F1: {deep_avg['f1_macro']:.4f}")
    print(f"  Improvement: {100*(deep_avg['accuracy']-classical_avg['accuracy']):.2f}% increase in accuracy")

## Summary and Key Findings

### Model Performance:
- **Best Model**: [Will be filled based on results]
- **Classical ML**: Generally lower performance but faster training
- **Deep Learning**: Higher accuracy with pretrained models

### Key Insights:
1. Transfer learning with pretrained models significantly improves performance
2. Data augmentation helps reduce overfitting
3. Larger image sizes (128x128) generally perform better than smaller sizes (64x64)
4. Some classes are consistently harder to classify across all models

### Recommendations:
- Use deep learning models (ResNet/ViT) for best accuracy
- Apply data augmentation for limited data scenarios
- Consider class imbalance when evaluating model performance